# Testing Laura Nguyen's InkML Converter Wrapper

This notebook tests the Python wrapper for Laura Nguyen's InkML converter.

In [ ]:
# Import necessary modules
import sys
import os
import tempfile
import shutil
from pathlib import Path

# Add the current directory to the path
sys.path.append('.')

# Import the converter class
from inkml_converter_wrapper import InkMLConverter

print("Modules imported successfully!")

In [ ]:
# Initialize the converter
try:
    converter = InkMLConverter()
    print("InkML converter initialized successfully!")
    print(f"Converter script path: {converter.converter_script}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Script directory exists: {Path(converter.converter_script).exists()}")
except Exception as e:
    print(f"Error initializing converter: {e}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Looking for converter script at: {os.path.abspath('convert_txt_to_inkml.ts')}")
    # List files in current directory
    print("Files in current directory:")
    for file in os.listdir('.'):  
        print(f"  - {file}")

In [ ]:
# Test 1: Single file conversion
print("Test 1: Single file conversion")

# Check if converter is available before proceeding
if 'converter' in globals():
    # Create a temporary TXT file with sample data
    sample_content = """Stroke 1
10.5 20.3
11.2 21.0
12.1 22.5

Stroke 2
15.0 25.0
16.5 26.2
17.8 27.9"""

    # Create a temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as temp_file:
        temp_file.write(sample_content)
        temp_txt_path = temp_file.name

    print(f"Created temporary TXT file: {temp_txt_path}")

    try:
        # Convert the file
        inkml_path = converter.convert_txt_to_inkml(temp_txt_path)
        print(f"Successfully converted to InkML: {inkml_path}")
        
        # Display the content of the generated InkML file
        with open(inkml_path, 'r') as f:
            inkml_content = f.read()
            print("\nGenerated InkML content:")
            print(inkml_content)
            
    except Exception as e:
        print(f"Error during conversion: {e}")
        
    # Clean up temporary files
    if os.path.exists(temp_txt_path):
        os.remove(temp_txt_path)
    # Note: inkml_path might not be defined if conversion failed
    try:
        if 'inkml_path' in locals() and os.path.exists(inkml_path):
            os.remove(inkml_path)
    except:
        pass
else:
    print("Converter not available, skipping this test.")

In [ ]:
# Test 2: String to InkML conversion
print("Test 2: String to InkML conversion")

# Check if converter is available before proceeding
if 'converter' in globals():
    sample_content = """Stroke 1
5.0 10.0
6.0 11.0
7.0 12.0

Stroke 2
15.0 20.0
16.0 21.0
17.0 22.0"""
    
    print("Original string content:")
    print(sample_content)
    print("\nConverting to InkML...")

    try:
        # Convert from string
        inkml_path = converter.convert_from_string(sample_content, "test_output")
        print(f"\nSuccessfully converted string to InkML: {inkml_path}")
        
        # Display the content of the generated InkML file
        with open(inkml_path, 'r') as f:
            inkml_content = f.read()
            print("\nGenerated InkML content:")
            print(inkml_content)
            
    except Exception as e:
        print(f"Error during string conversion: {e}")
        
    # Clean up
    try:
        if 'inkml_path' in locals() and os.path.exists(inkml_path):
            os.remove(inkml_path)
    except:
        pass
else:
    print("Converter not available, skipping this test.")

In [ ]:
# Test 3: Batch conversion
print("Test 3: Batch conversion")

# Check if converter is available before proceeding
if 'converter' in globals():
    # Create a temporary directory with multiple sample TXT files
    temp_dir = tempfile.mkdtemp()
    print(f"Created temporary directory: {temp_dir}")

    # Create multiple sample TXT files
    for i in range(3):
        sample_content = f"""Stroke 1
    {i*5}.0 {i*10}.0
    {i*5+1}.0 {i*10+1}.0
    {i*5+2}.0 {i*10+2}.0"""
        
        file_path = os.path.join(temp_dir, f"sample_{i}.txt")
        with open(file_path, 'w') as f:
            f.write(sample_content)
        print(f"Created sample file: {file_path}")

    try:
        # Perform batch conversion
        output_dir = os.path.join(temp_dir, "output")
        os.makedirs(output_dir, exist_ok=True)
        
        inkml_files = converter.batch_convert_directory(temp_dir, output_dir)
        print(f"\nSuccessfully converted {len(inkml_files)} files:")
        for inkml_file in inkml_files:
            print(f"  - {inkml_file}")
            
        # Display content of one of the generated files
        if inkml_files:
            with open(inkml_files[0], 'r') as f:
                content = f.read()
                print(f"\nContent of {os.path.basename(inkml_files[0])}:")
                print(content)
            
    except Exception as e:
        print(f"Error during batch conversion: {e}")
        
    # Clean up
    shutil.rmtree(temp_dir)
    print(f"\nCleaned up temporary directory: {temp_dir}")
else:
    print("Converter not available, skipping this test.")

In [ ]:
# Test 4: Error handling
print("Test 4: Error handling")

# Check if converter is available before proceeding
if 'converter' in globals():
    # Test with invalid file path
    try:
        invalid_path = "/invalid/path/file.txt"
        inkml_path = converter.convert_txt_to_inkml(invalid_path)
        print("Unexpectedly succeeded with invalid path")
    except Exception as e:
        print(f"Correctly caught error for invalid path: {e}")

    # Test with non-existent converter script
    try:
        fake_converter = InkMLConverter(converter_script_path="nonexistent_script.ts")
        sample_content = """Stroke 1
1.0 2.0
3.0 4.0"""
        inkml_path = fake_converter.convert_from_string(sample_content, "fake_test")
        print("Unexpectedly succeeded with nonexistent script")
    except Exception as e:
        print(f"Correctly caught error for nonexistent script: {e}")
else:
    print("Converter not available, skipping this test.")

In [ ]:
# Summary
print("\n=== TEST SUMMARY ===")
if 'converter' in globals():
    print("All tests completed. The Python wrapper for Laura Nguyen's InkML converter:")
    print("1. Successfully converts individual TXT files to InkML format")
    print("2. Converts string content to InkML format")
    print("3. Performs batch conversion of multiple files")
    print("4. Handles errors appropriately")
    print("\nThe wrapper is ready for integration with Flask applications.")
else:
    print("Tests could not be completed due to converter initialization failure.")
    print("Please ensure the convert_txt_to_inkml.ts file exists and Node.js is installed.")